In [123]:
/* // need this? 
  - like https://colab.research.google.com/gist/korakot/22abd6eccac229e9cb9a027b088b50d6/notebook.ipynb#scrollTo=ZVkF16NwcMsZ
  https://stackoverflow.com/a/60507269
  https://dev.to/obenjiro/silence-of-the-fans-part-1-javascript-quickstart-5f3m
!npm install kromosynth;
*/

In [124]:
require("esm-hook")
var {
  getNewAudioSynthesisGenome,
  getClassScoresForGenome,
  getAudioBufferFromGenomeAndMeta
} = require('kromosynth');
var { AudioContext, OfflineAudioContext } = require('node-web-audio-api');

[Function (anonymous)]

In [125]:
var evoParams = {
  "waveNetwork": {
    "neatParameters": { // see kromosynth > cppn-neat > network-evolution.js
      "pMutateAddConnection": 0.13,
      "pMutateAddNode": 0.13,
      "pMutateDeleteSimpleNeuron": 0.00,
      "pMutateDeleteConnection": 0.00,
      "pMutateConnectionWeights": 0.72,
      "pMutateChangeActivations": 0.02,

      "pNodeMutateActivationRate": 0.2,
      "connectionWeightRange": 3.0,
      "disallowRecurrence": true
    },
    "iecOptions": {
      "initialMutationCount": 5,
      "postMutationCount": 5  // AKA mutationsOnCreation
    },
    "activationFunctionProbabilities": {
      "triangle": 0.25,
      "sawtooth": 0.25,
      "StepFunction": 0.25,
      "Sine": 0.25,
      "Sine2": 0.25, // https://www.intmath.com/functions-and-graphs/graphs-using-svg.php?function1=sin(2*x)&function2=&xMin=-5&xMax=5&xGrid=2&yMin=-1&yMax=1.1&yGrid=1&gwidth=290&gheight=130

      "cos": 0,
      "arctan": 0,
      "spike": 0,

      "BipolarSigmoid": 0, // https://www.intmath.com/functions-and-graphs/graphs-using-svg.php?function1=(2.0+%2F+(1.0+%2B+exp(-4.9+*+x)))+-+1.0&function2=&xMin=-5&xMax=5&xGrid=2&yMin=-1&yMax=1&yGrid=1&gwidth=450&gheight=250
      "PlainSigmoid": 0, // https://www.intmath.com/functions-and-graphs/graphs-using-svg.php?function1=1.0%2F(1.0%2B(exp(-x)))&function2=&xMin=-5&xMax=5&xGrid=2&yMin=-1&yMax=1&yGrid=1&gwidth=450&gheight=250
      "Gaussian": 0, // https://www.intmath.com/functions-and-graphs/graphs-using-svg.php?function1=2+*+exp(-(x*2.5)%5E2)+-+1&function2=&xMin=-5&xMax=5&xGrid=2&yMin=-1&yMax=1&yGrid=1&gwidth=450&gheight=250
      "Linear": 0, // https://www.intmath.com/functions-and-graphs/graphs-using-svg.php?function1=abs(x)&function2=&xMin=-5&xMax=5&xGrid=2&yMin=-1&yMax=1&yGrid=1&gwidth=450&gheight=250
      "NullFn": 0 // https://www.intmath.com/functions-and-graphs/graphs-using-svg.php?function1=0&function2=&xMin=-5&xMax=5&xGrid=2&yMin=-1&yMax=1&yGrid=1&gwidth=450&gheight=250
    }
  },
  "audioGraph": { // aka asNEAT
    "mutationParams": {
      "mutationDistance": 0.5,

      // Chances must add up to 1.0
      "splitMutationChance": 0.2,
      "addOscillatorChance": 0.1,
      "addAudioBufferSourceChance": 0.1,
      "addPartialAndEnvelopeChance": 0.0,
      "addConnectionChance": 0.2,
      "mutateConnectionWeightsChance": 0.2, // 0.25,
      "mutateNodeParametersChance": 0.2 // 0.25
    },
    "defaultParameters": {
      "connectionMutationRate": [0.05, 0.8],

      "nodeMutationRate": [0.05, 0.8],

      // percentage of addOscillatorMutations will
      // generate a node for fm, as opposed to strict audio output
      "addOscillatorFMMutationRate": 0.1,

      // Percentage of addConnectionMutation will generate a connection
      // for fm, as opposed to a strict audio connection
      "addConnectionFMMutationRate": 0.5,

      // Chance between an Oscillator node or a (CPPN) network output node, when "addOscillator"
      "addOscillatorNetworkOutputVsOscillatorNodeRate": 0.5,
      
      // Rate between audio buffer source node, a wavetable node or additive synthesis node
      // addAudioBufferSource?Chances must add up to 1
      "addAudioBufferSourceProperChance": 0.3,
      "addAudioBufferSourceWavetableChance": 0.3,
      "addAudioBufferSourceAdditiveChance": 0.4,

      "includeNoise": true,
      
      // initial node chances must add up to 1
      "initialOscillatorChance": 0.5,
      "initialAudioBufferSourceChance": 0.5
    }
  }
}

var classScoringDurations = [4.0];
var classScoringNoteDeltas = [0];
var classScoringVelocities = [1];

In [126]:
var genome = getNewAudioSynthesisGenome(
  undefined, // evolutionRunId
  undefined, // generationNumber
  undefined, // parentIndex
  evoParams
);
console.log(genome);

{
  waveNetwork: {
    offspring: NeatGenome {
      gid: 0,
      fitness: 0,
      nodes: [Array],
      connections: [Array],
      mutations: [],
      debug: false,
      behavior: [Object],
      realFitness: 0,
      age: 0,
      localObjectivesCompetition: [],
      meta: {},
      inputNodeCount: 2,
      inputAndBiasNodeCount: 3,
      outputNodeCount: 18,
      inputBiasOutputNodeCount: 21,
      inputBiasOutputNodeCountMinus2: 19,
      networkAdaptable: false,
      networkModulatory: false,
      connectionLookup: null,
      nodeLookup: null,
      parent: null
    },
    parents: []
  },
  asNEATPatch: Network {
    nodes: [ [Node], [Node], [Node], [Node], [Node] ],
    connections: [ [Connection], [Connection], [Connection], [Connection] ],
    generation: 0,
    connectionMutationInterpolationType: 'exponential',
    connectionMutationRate: [ 0.05, 0.8 ],
    nodeMutationInterpolationType: 'exponential',
    nodeMutationRate: [ 0.05, 0.8 ],
    addOscillatorFMMutatio

In [127]:
var duration = 1;
var noteDelta = 0;
var velocity = 1;
var reverse = false;
var SAMPLE_RATE = 44100;


// const audioBuffer = await getAudioBufferFromGenomeAndMeta( ...

In [128]:
var offlineAudioContext = new OfflineAudioContext({
  numberOfChannels: 2,
  length: SAMPLE_RATE * duration,
  sampleRate: SAMPLE_RATE,
});

In [129]:
var audioContext = new AudioContext({sampleRate: SAMPLE_RATE});

In [130]:
var genomeAndMeta = {genome};
getAudioBufferFromGenomeAndMeta(
  genomeAndMeta,
  duration, noteDelta, velocity, reverse,
  false, // asDataArray
  offlineAudioContext,
  audioContext,
  true, //useOvertoneInharmonicityFactors
).then( audioBuffer => { 
  console.log(audioBuffer);
 });

 Activating network,
        for 2 unique periods
        and 44100 samples,
        took 73.04408299922943  milliseconds.


Promise { <pending> }

 instantiating audio context took 0.0005409717559814453 milliseconds
Wiring up audio graph...


In [131]:
getClassScoresForGenome(
  genome,
  Object.values(classScoringDurations ),
  Object.values( classScoringNoteDeltas ),
  Object.values( classScoringVelocities ),
  'yamnet', //classificationGraphModel, 
  undefined, //modelUrl,
  true, //useGpuForTensorflow,
  true // supplyAudioContextInstances
).then( scores => {
  console.log("mapElites -> getClassScoresForGenome: ", scores);
})
.catch( e => {
  console.error("mapElites -> getClassScoresForGenome: ", e);
  error = e;
} );

Done wiring up audio graph in 12.748667001724243 ms, will now render.
 Rendering audio graph took 0.7414590120315552 milliseconds


AudioBuffer {
  '0': NaN,
  '1': NaN,
  '2': NaN,
  '3': NaN,
  '4': NaN,
  '5': NaN,
  '6': NaN,
  '7': NaN,
  '8': NaN,
  '9': NaN,
  '10': NaN,
  '11': NaN,
  '12': NaN,
  '13': NaN,
  '14': NaN,
  '15': NaN,
  '16': NaN,
  '17': NaN,
  '18': NaN,
  '19': NaN,
  '20': NaN,
  '21': NaN,
  '22': NaN,
  '23': NaN,
  '24': NaN,
  '25': NaN,
  '26': NaN,
  '27': NaN,
  '28': NaN,
  '29': NaN,
  '30': NaN,
  '31': NaN,
  '32': NaN,
  '33': NaN,
  '34': NaN,
  '35': NaN,
  '36': NaN,
  '37': NaN,
  '38': NaN,
  '39': NaN,
  '40': NaN,
  '41': NaN,
  '42': NaN,
  '43': NaN,
  '44': NaN,
  '45': NaN,
  '46': NaN,
  '47': NaN,
  '48': NaN,
  '49': NaN,
  '50': NaN,
  '51': NaN,
  '52': NaN,
  '53': NaN,
  '54': NaN,
  '55': NaN,
  '56': NaN,
  '57': NaN,
  '58': NaN,
  '59': NaN,
  '60': NaN,
  '61': NaN,
  '62': NaN,
  '63': NaN,
  '64': NaN,
  '65': NaN,
  '66': NaN,
  '67': NaN,
  '68': NaN,
  '69': NaN,
  '70': NaN,
  '71': NaN,
  '72': NaN,
  '73': NaN,
  '74': NaN,
  '75': NaN,
  '76': 

Promise { <pending> }

 instantiating audio context took 0.0007089972496032715 milliseconds


Wiring up audio graph...
Done wiring up audio graph in 10.21262502670288 ms, will now render.
 Rendering audio graph took 1.040624976158142 milliseconds
audioData Float32Array(64000) [
                     0, 0.004811357241123915, 0.009622577577829361,
  0.014433631673455238, 0.019244495779275894, 0.024055130779743195,
  0.028865516185760498,  0.03367562219500542,  0.03848542273044586,
   0.04329488426446915,  0.04810398444533348,  0.05291270837187767,
   0.05772102251648903,  0.06252892315387726,  0.06733638048171997,
   0.07214341312646866,  0.07694996893405914,   0.0817561075091362,
   0.08656178414821625,  0.09136703610420227,  0.09617187827825546,
   0.10097634792327881,   0.1057804524898529,  0.11058425903320312,
   0.11538779735565186,  0.12019113451242447,  0.12499429285526276,
   0.12979735434055328,  0.13460038602352142,  0.13940346240997314,
   0.14420665800571442,  0.14901001751422882,   0.1538136601448059,
   0.15861757099628448,  0.16342194378376007,   0.1682267040014267,

In [132]:
// need this for running shell command
var { spawn } = require('child_process');
var sh = (cmd) => { 
    $$.async();
    var sp = spawn(cmd, { cwd: process.cwd(), stdio: 'pipe', shell: true, encoding: 'utf-8' });
    sp.stdout.on('data', data => console.log(data.toString()));
    sp.stderr.on('data', data => console.error(data.toString()));
    sp.on('close', () => $$.done());
};
sh('date +"%s"');

1682800331

